# Retrieving ground truth for equation images

In [1]:
import pandas as pd
gt_2013 = pd.read_csv('./gt_tsv/groundtruth_2013.tsv', sep='\t', names=['path', 'gt'])
gt_2014 = pd.read_csv('./gt_tsv/groundtruth_2014.tsv', sep='\t', names=['path', 'gt'])
gt_2016 = pd.read_csv('./gt_tsv/groundtruth_2016.tsv', sep='\t', names=['path', 'gt'])

In [2]:
combined_df = pd.concat([gt_2013, gt_2014, gt_2016], ignore_index=True)

In [5]:
import os
import glob

class_directories = ["equation_dataset"]

image_extensions = ['.jpg', '.png', '.jpeg']
image_file_names = []

for directory in class_directories:
    for ext in image_extensions:
        image_files = glob.glob(os.path.join(directory, f"*{ext}"))
        image_file_names.extend([os.path.splitext(os.path.basename(file))[0] for file in image_files])

print(image_file_names)

['120_em_300', '127_em_508', '18_em_10', '18_em_21', '20_em_42', '20_em_47', '23_em_56', '26_em_78', '26_em_93', '27_em_110', '27_em_119', '29_em_159', '31_em_187', '31_em_195', '34_em_227', '35_em_0', '35_em_15', '35_em_19', '35_em_6', '36_em_28', '502_em_0', '502_em_2', '502_em_23', '504_em_44', '505_em_47', '507_em_76', '511_em_254', '512_em_280', '512_em_284', '513_em_310', '515_em_351', '516_em_376', '518_em_428', 'ritm_422_0', 'RIT_2014_122', 'RIT_2014_130', 'RIT_2014_156', 'RIT_2014_204', 'RIT_2014_246', 'RIT_2014_272', 'RIT_2014_284', 'RIT_2014_29', 'RIT_2014_299', 'RIT_2014_57', 'RIT_2014_70', 'RIT_2014_95', 'RIT_2014_97', 'rit_420_3', 'rit_42150_0', 'rit_4220_1', 'rit_4225_4', 'rit_4240_0', 'rit_4265_0', 'rit_4295_0', 'UN_101_em_5', 'UN_105_em_109', 'UN_110_em_227', 'UN_110_em_229', 'UN_111_em_259', 'UN_112_em_282', 'UN_112_em_283', 'UN_114_em_304', 'UN_115_em_316', 'UN_117_em_346', 'UN_117_em_352', 'UN_117_em_357', 'UN_118_em_385', 'UN_119_em_400', 'UN_120_em_428', 'UN_121_e

In [6]:
print(len(image_file_names))

102


In [7]:
combined_df

,path,gt
0,120_em_292,(y-y^{calc})
1,118_em_221,2sp_{z}
2,rit_4235_3,\frac {5} {6}
3,rit_42160_3,\frac {2} {\frac {1} {x} + \frac {1} {y}}
4,116_em_162,L(w)=\frac{m}{2}||w||^2-V(r)
...,...,...
2799,UN_127_em_593,\lim_{r\rightarrow \infty} f(r) = 0
2800,UN_127_em_597,9\mbox{x}9
2801,UN_464_em_928,\frac{T}{L} \log \frac{T}{L}
2802,UN_116_em_335,\frac 1x


In [9]:
import numpy as np

def testing(path, image_file_names):
    if path in image_file_names:
        return path

    return np.nan

combined_df["path"] = combined_df["path"].apply(testing, args=(image_file_names,))

filtered_df = combined_df.dropna()

In [15]:
filtered_df.to_csv("./ground_truth_csv/final_gt.csv", index=False)